In [164]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore')

In [165]:
df_athletes_activities = pd.read_csv('C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/scripts_prediction_model/final/st1_df_athletes_activities.csv', parse_dates=['activity_date'])
df_athletes_activities

,athlete_id,activity_date,total_distance (km),total_time (min),pace (min/km),speed (km/h)
0,1,2020-07-09,6.03,39.0,6.47,9.28
1,1,2020-07-10,25.35,140.0,5.52,10.86
2,1,2020-07-12,7.57,55.0,7.27,8.26
3,1,2020-07-16,3.61,23.0,6.37,9.42
4,1,2020-07-20,25.66,127.0,4.95,12.12
...,...,...,...,...,...,...
903,7,2024-06-16,10.06,45.0,4.47,13.26
904,7,2024-06-18,5.20,33.0,6.35,9.27
905,7,2024-06-21,12.10,62.0,5.12,11.71
906,7,2024-06-26,4.08,22.0,5.39,11.13


In [166]:
df_athletes_activities_new = pd.DataFrame()

for id, df_grouped in df_athletes_activities.groupby('athlete_id'):

    df_grouped.insert(1, 'is_activity', 'Activity')

    # Data da primeira e última atividade para criar um range
    start, end = df_grouped['activity_date'].iloc[0], df_grouped['activity_date'].iloc[-1]

    # Dias entre a primeira e última atividade
    dates = pd.date_range(start, end, freq='d')

    # Colocando todas as datas em um df
    df_dates = pd.DataFrame({'activity_date': dates})

    # Right join pra poder manter os dados das atividades existentes mas adicionar linhas para os novos dias
    df_grouped = df_dates.merge(df_grouped, on='activity_date', how='left')

    # Preechendo os valores nulos como Dia sem atividade
    df_grouped['is_activity'].fillna('No Activity', inplace=True)
    
    # Preenchendo novamente a coluna id para evitar NaN das linhas sem atividade
    df_grouped['athlete_id'] = id

    # Concatenação dos valores de cada atleta por id a um Dataframe
    df_athletes_activities_new= pd.concat([df_athletes_activities_new, df_grouped])

df_athletes_activities_new['is_activity'] = (df_athletes_activities_new['is_activity'] == 'Activity') * 1
df_athletes_activities_new.reset_index(drop=True, inplace=True)
df_athletes_activities_new

,activity_date,athlete_id,is_activity,total_distance (km),total_time (min),pace (min/km),speed (km/h)
0,2020-07-09,1,1,6.03,39.0,6.47,9.28
1,2020-07-10,1,1,25.35,140.0,5.52,10.86
2,2020-07-11,1,0,NaN,NaN,NaN,NaN
3,2020-07-12,1,1,7.57,55.0,7.27,8.26
4,2020-07-13,1,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
4623,2024-06-24,7,0,NaN,NaN,NaN,NaN
4624,2024-06-25,7,0,NaN,NaN,NaN,NaN
4625,2024-06-26,7,1,4.08,22.0,5.39,11.13
4626,2024-06-27,7,0,NaN,NaN,NaN,NaN


In [167]:
# Número de atividades dentro desse período de dias em uma janela deslizante, por atleta
# Quantidade de atividades num período específico de dias
activity_days = 7
df_athletes_activities_new['weekly_frequency'] = [values for values in df_athletes_activities_new.groupby('athlete_id')['is_activity'].rolling(activity_days).sum()]
df_athletes_activities_new.dropna(subset= 'weekly_frequency', inplace= True)
df_athletes_activities_new['weekly_frequency'] = df_athletes_activities_new['weekly_frequency'].astype(int)

In [168]:
# Dropando as linhas que não atividades
df_athletes_activities_new.drop(df_athletes_activities_new[df_athletes_activities_new['is_activity'] == 0].index, axis=0, inplace=True)
# Dropando as coluna que checam se é ou não atividade e se é ou não corrida (não precisa mais)
df_athletes_activities_new.drop(['is_activity'], axis=1, inplace= True)
# Resetando index depois do drop
df_athletes_activities_new.reset_index(drop=True, inplace=True)

In [169]:
# Fazendo a diferença entre a atividade seguinte e a anterior e preenchendo a primeira linha com 0
df_athletes_activities_new['days_since_last_activity'] = (df_athletes_activities_new.groupby('athlete_id')['activity_date'].diff().dt.days.fillna(0)).astype(int)

In [170]:
df_athletes_activities_new['will_churn'] = (((df_athletes_activities_new['days_since_last_activity'] >= 14) * 1).shift(-1).fillna(0)).astype(int)
df_athletes_activities_new

,activity_date,athlete_id,total_distance (km),total_time (min),pace (min/km),speed (km/h),weekly_frequency,days_since_last_activity,will_churn
0,2020-07-16,1,3.61,23.0,6.37,9.42,3,0,0
1,2020-07-20,1,25.66,127.0,4.95,12.12,2,4,0
2,2020-07-22,1,25.54,149.0,5.83,10.28,3,2,0
3,2020-07-24,1,5.84,41.0,7.02,8.55,3,2,0
4,2020-07-25,1,4.28,32.0,7.48,8.02,4,1,0
...,...,...,...,...,...,...,...,...,...
885,2024-06-16,7,10.06,45.0,4.47,13.26,3,3,0
886,2024-06-18,7,5.20,33.0,6.35,9.27,3,2,0
887,2024-06-21,7,12.10,62.0,5.12,11.71,3,3,0
888,2024-06-26,7,4.08,22.0,5.39,11.13,2,5,0


In [171]:
df_athletes_activities_new.groupby('athlete_id')['will_churn'].value_counts()

athlete_id  will_churn
1           0             101
            1              10
2           0              50
            1              11
3           0              92
            1               2
4           0             126
            1              22
5           0             342
            1              10
6           0              79
            1               1
7           0              44
Name: count, dtype: int64

In [172]:
df_athletes_activities_new['will_churn'].value_counts()

will_churn
0    834
1     56
Name: count, dtype: int64

In [174]:
df_athletes_activities_new.to_csv('C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/scripts_prediction_model/final/st2_df_athletes_activities.csv', index = False)